In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype # 데이터프레임에서 범주형 데이터(categorical data)의 데이터 타입을 정의할 때 사용합니다.
pd.set_option('mode.chained_assignment',None) 

# 1. Covid19 원본 데이터셋 로딩
df_covid19 = pd.read_csv("owid-covid-data.csv")

# 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100생성
df_covid19['date'] = pd.to_datetime(df_covid19['date'],format='%Y-%m-%d')

# 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR',"OWID_ASI",'OWID_EUR',"OWID_OCE","OWID_NAM",'OWID_SAM','OWID_AFR'])) & (df_covid19['date']>=(max(df_covid19['date'] - timedelta(days=100))))]

# 대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea',"location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia',"location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe',"location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania',"location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America',"location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America',"location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa',"location"] = '아프리카'

# 이산형 변수 설정
ord = CategoricalDtype(categories=['한국','아시아','유럽','북미','남미','아프리카','오세아니아'],ordered=True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

# date로 정리
df_covid19_100 = df_covid19_100.sort_values(by='date')

# 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은 형태의 데이터프레임 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date','location','new_cases','people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자','people_fully_vaccinated_per_hundred':'백신접종완료자'})

display(df_covid19_100_wide.head())

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date',columns='location',values=['확진자','백신접종완료자']).sort_values(by='date')
display(df_covid19_100_wide.head())

df_covid19_100_wide.columns = ['확진자_한국','확진자_아시아','확진자_유럽','확진자_북미','확진자_남미','확진자_아프리카','확진자_오세아니아','백신접종완료자_한국',
                               '백신접종완료자_아시아','백신접종완료자_유럽','백신접종완료자_북미','백신접종완료자_남미','백신접종완료자_아프리카','백신접종완료자_오세아니아']

# 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code','continent','location'],dropna=True).agg(
    인구수 = ('population','max'),
    전체사망자수 = ('new_deaths','sum'),
    백신접종자완료자수 = ('people_fully_vaccinated','max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred','max'),    
    인구백명당부스터접종자수 = ('total_boosters_per_hundred','max')).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / df_covid19_stat['인구수'] * 10000,5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / df_covid19_stat['인구수'] 

## 여백 설정을 위한 변수 설정
margins_P = {'t':50,'b':25,'1':25,'r':25}

,date,location,확진자,백신접종완료자
2016,2022-10-03,아프리카,754.0,22.76
163005,2022-10-03,북미,45451.0,64.43
207753,2022-10-03,남미,15450.0,76.48
12551,2022-10-03,아시아,84967.0,71.77
208839,2022-10-03,한국,16423.0,85.73


확진자                                                         \
location         한국       아시아        유럽       북미       남미    아프리카    오세아니아   
date                                                                         
2022-10-03  16423.0   84967.0  168183.0  45451.0  15450.0   754.0   5145.0   
2022-10-04  34710.0  149537.0  384493.0  61078.0   2379.0  1217.0   1536.0   
2022-10-05  28603.0  149638.0  330710.0  80045.0  18412.0   780.0   3098.0   
2022-10-06  22259.0  127116.0  394838.0  71088.0  14690.0   904.0  31919.0   
2022-10-07  19379.0  124355.0  375601.0  67185.0   4412.0   533.0   2517.0   

           백신접종완료자                                            
location        한국    아시아     유럽     북미     남미   아프리카  오세아니아  
date                                                          
2022-10-03   85.73  71.77  66.58  64.43  76.48  22.76  61.67  
2022-10-04   85.73  71.78  66.62  64.44  76.48  22.76  61.67  
2022-10-05   85.73  71.80  66.63  64.45  76.48  22.76  61.68  
2022-10-06   85.73  71.80  66.63  64.46  76.48  22.76  61.68  
2022-10-07   85.73  71.81  66.63  64.79  76.53  22.76  61.68